In [33]:
%rest -f

UsageError: Line magic function `%rest` not found.


# USCIS i-485 analysis 
This is a test code to pre process the following websites. 

https://www.immihelp.com/i-485-tracker/
https://www.trackitt.com/usa-immigration-trackers/i485-eb

The above website shows the code, let's try to download them.

Steps
1. Analyze the website and see how the data is stored
2. use request and bs4 to download data if possible 
3. save it in excel

Step 1: Analyze the website and see how the data is stored 

Why: This step is important becaues without understanding the structure of the HTML code we would not be able to parse it by beautiful soup. 
The structure of this website is that the data is stored in the class called "c-Sticky-table" and structure is as follows: 
c-Sticky-table
- thead - tr - <th> text </th>
- tbody 
  - tr - <td> text </td>
  - tr - <td> text </td> .... 

Based on the above, it is good to perform the following
1. Obtain the header by parsing the html as table>thead>th 
2. Obtain the data by iterating each tr and collect the td from the text 

# Web scraping portion

<h>Import necessary libraries</h>

In [34]:
# standard library 
import requests
import csv, re
import time 
import cProfile
import pstats

# external library 
from bs4 import BeautifulSoup
from tqdm import tqdm 


In [35]:
# declare infos
base_url = "https://www.immihelp.com/i-485-tracker/"


In [36]:
def get_last_page(session, url):
    """This function takes session and url as an argument and return the last page number of the url. 

    Args:
        session (request.session): session object generated by request
        url (string): target url 

    Returns:
        last_page (int): last page number of the table 
    """
    soup = get_soup(session, url)
    # Find how many pages are there
    last_page_url = soup.find(title="Last Page", href=True)
    # This will result in "/i-485-tracker/100/"
    last_page_url = last_page_url['href']
    # Remove the leading and trailing "/"
    last_page_url =last_page_url.strip('/')
    # Split by "/" delimiter
    last_page_url = last_page_url.split('/')
    # the last item is the page number
    last_page_num = last_page_url[-1]
    # It is string, hence convert to integer
    last_page_num = int(last_page_num)
    return last_page_num

In [37]:
def get_header(header_table):
    """This function takes the target table and return header list 

    Args:
        header_table (table object from bs4): 

    Returns:
        list: header list
    """
    headers = []
    # For each html tag "th" in the table, append the text inside it. This will obtain the header of the table 
    for col in header_table.find_all("th"):
        headers.append(col.text)
    return headers

In [38]:
def get_table(session, url, target):
    soup = get_soup(session, url)
    table = soup.find(class_=target)
    return table

In [39]:
def get_soup(session, url):
    response = session.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    return soup

In [40]:
def get_data():
    # iterate through pages version 
    with requests.Session() as s: 
        data =[]
        last_page = get_last_page(s, base_url)
        
        target_table = "c-Sticky-table"
        table = get_table(s, base_url,target_table)
        # Initialize headers list 
        headers = get_header(table.thead)
        last_page = 3
        # For each page 
        for page in range(1,last_page+1):
            # Initialize body table and raw data list 
            current_page = base_url + f"{page}/"
            table = get_table(s,current_page,target_table)
            body_table = table.tbody
            
            # For each row in the body of the table 
            for row in body_table.find_all("tr"):
                t_row ={}
                # For each col of the row 
                for col,header in zip(row.find_all("td"),headers):
                    # print(f"printing column {col.text}")
                    # Store them in dictionary, header is the key and col is the data. make sure to remove "\n" and extra spaces
                    t_row[header] = col.text.replace("\n","").strip()
                # print(t_row)
                data.append(t_row) 
    return data

In [41]:
def to_csv(data):
    keys = data[0].keys()
    file_name = 'test.csv'
    with open(file_name, 'w', encoding='utf-8', newline="") as data_file:
        dict_writer = csv.DictWriter(data_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(data) 

In [42]:
def main():
    data = get_data()
    to_csv(data)
# profile = cProfile.Profile()
# profile.runcall(get_data)
# ps = pstats.Stats(profile)
# ps.print_stats()

In [43]:
if __name__ == "__main__":
    main()

# Data analysis portion

In [44]:
import pandas as pd
import numpy as np 

In [45]:
df = pd.read_csv("test.csv")
df = df.replace("--",np.nan)
df.head()

,Posted by,Comments,I-485 Status,Citizenship Country,I-485 Wait Time,EAD Approval Wait Time,AP Approval Wait Time,Greencard Processing Time,I-485 Status Date,Category,...,EAD Approval Date,AP Applied,AP Application Date,AP Receipt Number,AP Approval Date,Most Recent LUD,Date Added,Last Updated,Notes,\nTracker Actions\n
0,calabaza,0 Comments,Approved,Colombia,NaN,NaN,NaN,777 Days,11/17/2021,IR1,...,NaN,NaN,NaN,NaN,NaN,NaN,{{'2021-05-26 17:54:30.0' | formatDateWithZone}},{{'2021-11-17 21:14:03.0' | formatDateWithZone}},NaN,NaN
1,anmomo0000,0 Comments,Pending,China,89 Days,NaN,NaN,1005 Days,09/29/2021,EB3,...,NaN,Yes,09/28/2021,NaN,NaN,NaN,{{'2021-09-29 15:51:48.0' | formatDateWithZone}},{{'2021-11-09 13:20:45.0' | formatDateWithZone}},NaN,NaN
2,aruna.apsangi,0 Comments,Pending,India,255 Days,NaN,NaN,173 Days,05/04/2021,EB1C,...,NaN,NaN,NaN,NaN,NaN,NaN,{{'2021-05-04 22:59:13.0' | formatDateWithZone}},{{'2021-11-02 12:35:29.0' | formatDateWithZone}},I485J - Receipt Notice - 10/15/21 received on ...,NaN
3,vrp0001,0 Comments,Approved,India,NaN,283 Days,283 Days,1236 Days,10/05/2021,EB1B,...,08/03/2021,Yes,10/24/2020,NaN,08/03/2021,NaN,{{'2021-10-06 21:52:18.0' | formatDateWithZone}},{{'2021-10-14 20:42:20.0' | formatDateWithZone}},NaN,NaN
4,mkrishn2,0 Comments,Approved,India,NaN,NaN,NaN,913 Days,06/11/2021,EB1C,...,NaN,Yes,04/06/2021,MSC2191140xxx,NaN,08/31/2021,{{'2021-09-07 19:04:35.0' | formatDateWithZone}},{{'2021-09-13 16:20:58.0' | formatDateWithZone}},Recv'd 485J receipt notice - 8/31/2021485 New ...,NaN


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 52 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Posted by                        30 non-null     object 
 1   Comments                         30 non-null     object 
 2   I-485 Status                     30 non-null     object 
 3   Citizenship Country              30 non-null     object 
 4   I-485 Wait Time                  21 non-null     object 
 5   EAD Approval Wait Time           10 non-null     object 
 6   AP Approval Wait Time            8 non-null      object 
 7   Greencard Processing Time        15 non-null     object 
 8   I-485 Status Date                15 non-null     object 
 9   Category                         30 non-null     object 
 10  Chargeability Country            30 non-null     object 
 11  Applicant Type                   26 non-null     object 
 12  Service Center          

In [47]:
df["Greencard Processing Time"] = df["Greencard Processing Time"].astype(str)
split_function = lambda x: x.split()[0]
df["Greencard Processing Time"] = df["Greencard Processing Time"].apply(split_function)
# df["Greencard Processing Time"]


In [48]:
df["Greencard Processing Time"].astype(float).describe()

count      15.000000
mean     1217.266667
std      1365.104678
min        43.000000
25%       364.000000
50%       777.000000
75%      1120.500000
max      4084.000000
Name: Greencard Processing Time, dtype: float64